In [49]:
import pandas as pd
import json

# チームIDをチーム名に変換する辞書作成
team_df = pd.read_csv('/Users/onehe/Desktop/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/team.csv')
team_dic = dict(zip(team_df['team_id'], team_df['team_name']))


# ○: フル出場, ▲: 途中出場, ▽: 途中退場, ◆: 途中出場途中退場, ×: 出場停止, ※: 他大会の影響で出場停止, B: ベンチ入り
# 各々の回数をカウント
tmp = ['rat_full_play', 'rat_out_play', 'rat_in_play', 'rat_inout_play', 'rat_bench_play', 'rat_susp_play']
colnames = ['rat_full_play', 'rat_out_play', 'rat_in_play', 'rat_inout_play', 'rat_bench_play', 'rat_susp_play']
colnames += [i + '_first' for i in tmp]
colnames += [i + '_second' for i in tmp]
colnames += [i + '_third' for i in tmp]
colnames += [i + '_forth' for i in tmp]
characters = ['○', '▲', '▽', '◆', 'B', '×'] * 5

# 2015 ~ 2018シーズンのjsonをcsvに変換
for year in range(2015, 2019):
    dfs = []
    team_df_year = team_df[team_df.year == year]
    for div, team_id in zip(team_df_year["div"], team_df_year["team_id"]):
        # 2stageかどうか
        is_2stage = year < 2017 and div == "J1"

        # 後で１試合あたりの指標に変換するために試合数の情報を記憶
        if div == 'J1':
            num_match = 34
        else:
            num_match = 42

        if is_2stage:
            with open(f"/Users/onehe/Desktop/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/event/event_play_{year}_{div}_{team_id}_s1.json", encoding="utf-8_sig") as f:
                first_dic = json.load(f)
            with open(f"/Users/onehe/Desktop/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/event/event_play_{year}_{div}_{team_id}_s2.json", encoding="utf-8_sig") as f:
                second_dic = json.load(f)

            first_df = pd.DataFrame.from_dict(first_dic, orient='index')
            second_df = pd.DataFrame.from_dict(second_dic, columns=list(range(17, 34)), orient='index')
            #(first_df)
            full_df = pd.concat([first_df, second_df], axis=1, join='outer')
            first_df = full_df.iloc[:, :num_match // 2]
            #print(first_df)
            second_df = full_df.iloc[:, num_match // 2:]
        else:
            with open(f"/Users/onehe/Desktop/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/event/event_play_{year}_{div}_{team_id}.json", encoding="utf-8_sig") as f:
                dic = json.load(f)

            full_df = pd.DataFrame.from_dict(dic, orient='index')

            # シーズンを四分割して集計するためにdataframeを分割
            num_to_div_list = [(num_match//4)*n for n in range(1,5)]
            #list_match = {f"match_{i}" for i in range(1,5)}
            #dict_match = dict(zip(num_to_div_list, list_match))
            
        first_df = full_df.iloc[:, :num_to_div_list[0]]
        second_df = full_df.iloc[:, num_to_div_list[0]:num_to_div_list[1]]
        third_df = full_df.iloc[:, num_to_div_list[1]:num_to_div_list[2]]
        forth_df = full_df.iloc[:, num_to_div_list[2]:]
            
        data = []
        for colname, ch in zip(colnames, characters):
            if 'first' in colname:
                data.append(first_df.apply(lambda d: d.str.contains(ch)).sum(axis=1).values)
            elif 'second' in colname:
                data.append(second_df.apply(lambda d: d.str.contains(ch)).sum(axis=1).values)
            elif 'third' in colname:
                data.append(third_df.apply(lambda d: d.str.contains(ch)).sum(axis=1).values)
            elif 'forth' in colname:
                data.append(forth_df.apply(lambda d: d.str.contains(ch)).sum(axis=1).values)
            else:
                data.append(full_df.apply(lambda d: d.str.contains(ch)).sum(axis=1).values)
        # 1試合あたりの値に変換
        df = pd.DataFrame(data).T
        df /= num_match
        df.columns = colnames
        df['team_id'] = team_id
        df['team'] = df['team_id'].map(int).map(team_dic)
        df.drop(['team_id'], axis=1, inplace=True)
        df['name'] = full_df.index

        dfs.append(df)

    df = pd.concat(dfs)
    df.to_csv(f'/Users/onehe/Desktop/コンペティション/Jリーグ選手出場時間予測_nishika/data/output/event_play_{year}_4div.csv', index=False)
    


In [36]:
forth_df.head(3)

,30,31,32,33,34,35,36,37,38,39,40,41
本間 幸司,B,B,B,B,B,B,B,B,,B,B,B
田向 泰輝,○,○,○,▽82,,○,○,○,○,○,○,○
浜崎 拓磨,B,B,B,▲8,○,,,,,B,B,B


In [37]:
num_to_div_list = [(num_match//4)*n for n in range(1,5)]
list_match = {f"match_{i}" for i in range(1,5)}
dict_match = dict(zip(num_to_div_list, list_match))

In [38]:
num_to_div_list

[10, 20, 30, 40]

In [40]:
list_match

{'match_1', 'match_2', 'match_3', 'match_4'}

In [41]:
dict_match

{10: 'match_3', 20: 'match_4', 30: 'match_1', 40: 'match_2'}

In [50]:
df.head

<bound method NDFrame.head of     rat_full_play  rat_out_play  rat_in_play  rat_inout_play  rat_bench_play  \
0        0.794118      0.000000     0.000000             0.0        0.058824   
1        0.029412      0.147059     0.176471             0.0        0.205882   
2        0.411765      0.000000     0.058824             0.0        0.088235   
3        0.470588      0.088235     0.029412             0.0        0.147059   
4        0.411765      0.000000     0.000000             0.0        0.000000   
..            ...           ...          ...             ...             ...   
30       0.000000      0.023810     0.000000             0.0        0.000000   
31       0.095238      0.333333     0.380952             0.0        0.047619   
32       0.000000      0.214286     0.071429             0.0        0.023810   
33       0.000000      0.166667     0.047619             0.0        0.214286   
34       0.690476      0.000000     0.000000             0.0        0.000000   

    rat_s

In [46]:
full_df

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
本間 幸司,B,B,B,B,B,B,○,○,○,○,...,B,B,B,B,B,B,,B,B,B
田向 泰輝,○,○,○,○,▽14,,,,,,...,○,▽82,,○,○,○,○,○,○,○
浜崎 拓磨,○,▽75,,,,,,,,,...,B,▲8,○,,,,,B,B,B
ンドカ ボニフェイス,,,▲4,▲6,B,B,B,B,,,...,,,,,B,,,,,
伊藤 槙人,B,B,B,B,B,○,○,○,○,○,...,○,○,○,○,○,○,○,○,○,○
ジエゴ,▲7,▲15,○,○,○,○,○,○,○,○,...,○,○,○,▽89,×,○,○,○,○,
船谷 圭祐,,,,,,,,,,,...,,,,,▲14,▲19,B,,,▲9
前 寛之,,,,,,,,,,,...,○,○,○,○,○,○,○,○,▽65,○
ジェフェルソン バイアーノ,▽65,▽80,▽76,▽69,▲20,▽87,▽77,▽72,▽80,▽45,...,▽67,○,▽63,▽54,▲20,○,×,▽81,▽57,▲21
木村 祐志,▽83,▽86,○,▽52,,○,○,▽79,○,▽60,...,○,▽82,▽63,○,○,▽66,○,○,○,○
